# Thalamic nuclei mean kurtosis estimation

> Import ROIs

In [13]:
import pandas as pd
import os
import re
import numpy as np
import nibabel as nb
import argparse
import textwrap

> Set input subjects / file locations

In [4]:
dataLoc = os.getcwd()
subjects = [x for x in os.listdir(dataLoc) if x.startswith('N') or x.startswith('F')]

In [34]:
def get_segment_map(subject,side):
    '''
    Returns nibabel matrix of 
    the connectivity-based segmentation map of thalamus
    '''
    segment_file = os.path.join(dataLoc,
                              subject,
                              'segmentation',
                              side,
                              'dki_biggest.nii.gz')
    f = nb.load(segment_file)
    segment_map = f.get_data()
    return segment_map

In [35]:
def get_mk_map(subject):
    '''
    Returns nibabel matrix of
    the whole brain mean kurtosis image
    '''
    mk_file = os.path.join(dataLoc,
                         subject,
                         'DKI',
                         'kmean.nii')
    f = nb.load(mk_file)
    mk_map = f.get_data()
    return mk_map

In [59]:
def get_mean_MK_for_nuclei(segment_map, mk_map, cortex):
    '''
    Returns mean of the MK 
    of the thalamic segments 
    in connection to sepcific cortex
    '''
    nuclei_dict = {"LPFC":1,
                  "LTC":2,
                  "MPFC":3,
                  "MTC":4,
                  "OCC":5,
                  "OFC":6,
                  "PC":7,
                  "SMC":8}
    nucleiNum = nuclei_dict[cortex]
    cortexMap = segment_map == nucleiNum
    segment_volume = np.sum(cortexMap)
    
    segment_mk = cortexMap * mk_map
    mk_mean = segment_mk[segment_mk!=0].mean()

    return mk_mean, segment_volume

In [60]:
def thalamic_mk(subject, side):
    '''
    Returns a pandas dataframe,
    containing MK of thalamic segments 
    for a given subject
    '''
    group = subject[:3]
    segment_map = get_segment_map(subject, side)
    mk_map = get_mk_map(subject)
    
    df = pd.DataFrame()
    for cortex in ["LPFC", "LTC", "MPFC", "MTC", "OCC", "OFC", "PC", "SMC"]:
        mk_mean, segment_volume = get_mean_MK_for_nuclei(segment_map, mk_map, cortex)
        cortex_df = pd.DataFrame(data = [[mk_mean, segment_volume,cortex, side, group]], 
                                 index=[subject], 
                                 columns=['MK','volume','cortex','side','group'])
        df = pd.concat([df, cortex_df])
    
    return df

In [61]:
thalamic_mk('NOR26_YJH','left')

,MK,volume,cortex,side,group
NOR26_YJH,0.830991446972,137,LPFC,left,NOR
NOR26_YJH,0.722406148911,128,LTC,left,NOR
NOR26_YJH,0.741360068321,71,MPFC,left,NOR
NOR26_YJH,0.724031865597,6,MTC,left,NOR
NOR26_YJH,nan,0,OCC,left,NOR
NOR26_YJH,1.07476615906,10,OFC,left,NOR
NOR26_YJH,0.856482565403,138,PC,left,NOR
NOR26_YJH,0.787861406803,126,SMC,left,NOR


In [62]:
thalamic_mk('NOR81_KSA','left')

,MK,volume,cortex,side,group
NOR81_KSA,0.973071753979,117,LPFC,left,NOR
NOR81_KSA,0.670243799686,1,LTC,left,NOR
NOR81_KSA,1.10995388031,6,MPFC,left,NOR
NOR81_KSA,0.633870244026,151,MTC,left,NOR
NOR81_KSA,nan,0,OCC,left,NOR
NOR81_KSA,1.1396958828,80,OFC,left,NOR
NOR81_KSA,0.956947147846,209,PC,left,NOR
NOR81_KSA,0.766120910645,30,SMC,left,NOR


In [42]:
def main():
    '''
    Estimates the mean kurtosis in the thalamic segments
    for all subjects.
    The thalamic segments is obtained from the connectivity-based segmentation
    from FSL.
    
    It returns mk_data.csv containing all subjects' data.
    
    '''
    allData = pd.DataFrame()
    for subject in subjects:
        for side in ['left','right']:
            subjectDf = thalamic_mk(subject, side)
            allData = pd.concat([allData, subjectDf])
    
    allData.to_csv('mk_data.csv')

```
if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter,
        description=textwrap.dedent('''\
            {codeName} : Estimates mean kurtosis in the thalamic nuclei
            ===========================================================
            '''.format(codeName=os.path.basename(__file__))))

    parser.add_argument(
        '-s', '--subjects',
        help='subject list',
        nargs='+')

    parser.add_argument(
        '-side', '--side',
        help='side')
    
    args = parser.parse_args()    
    thalamic_mk(args)
```